<a href="https://colab.research.google.com/github/jcsh4326/notebook/blob/master/TensorFlow_02_Use_TensorBoard_With_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 在colab中使用TensorBoard
利用ngrok进行内网穿透，访问TensorBoard

详见[stackoverflow](https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab)

In [0]:
#@title import
import numpy as np
import tensorflow as tf

In [0]:
#@title function helper
#@markdown edit layer <br> outputs = inputs * Weights + biases
def add_layer(inputs, in_size, out_size, activation_function=None):
  # add one more layer and return the output of this layer    
  with tf.name_scope('layer'):
    with tf.name_scope('weights'):
      Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='weights')
    with tf.name_scope('biases'):
      biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='biases')
    with tf.name_scope('outpus_y'):
      Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
    if activation_function is None:
      outputs = Wx_plus_b
    else:
      outputs = activation_function(Wx_plus_b, )
    return outputs

In [0]:
#@title define placeholder for inputs to network
with tf.name_scope('inputs'):
  # define placeholder for inputs to network
  xs = tf.placeholder(tf.float32, [None, 1], name='x_in')
  ys = tf.placeholder(tf.float32, [None, 1], name='y_in') 

In [0]:
#@title add hidden layer
# add hidden layer
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)
# add output layer
prediction = add_layer(l1, 10, 1, activation_function=None)

In [0]:
#@title loss function, the error between prediciton and real data
with tf.name_scope('loss'):
  loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), 1))   

In [0]:
#@title train
with tf.name_scope('train'):
  train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [0]:
#@title run
sess = tf.Session()

writer = tf.summary.FileWriter("logs/", sess.graph)
init = tf.global_variables_initializer()
sess.run(init)

In [0]:
#@title install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


--2018-11-26 08:55:07--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.152.127.232, 52.72.245.79, 52.72.145.109, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.152.127.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  7.03MB/s    in 0.7s    

2018-11-26 08:55:08 (7.03 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
#@title ngrok
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://1b6d980f.ngrok.io
